### Web scrape valid surf rides from the Smartfin website: 

In [57]:
#Imports 
import requests
import lxml.html as lh
import pandas as pd
import numpy as np
import time
from bs4 import BeautifulSoup
import re
import time

print("Done.")

Done.


In [58]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options

chrome_options = Options()
chrome_options.add_argument("--headless")

service = Service('/Users/jasminesimmons/Smartfin/Smartfin/Mini_Projects/ML_Wave_Stats/chromedriver')
service.start()
#driver = webdriver.Remote(service.service_url)
#driver.get('http://www.google.com/');
#time.sleep(5) # Let the user actually see something!
#driver.quit()

### Scrape verified surf sessions into a dataframe (can add more later):

In [59]:
# Start by just looking at my surf sessions in SD (near CDIP buoy): 

# 15218 - First VIRB filmed session - Oct. 24, 2018
# 15669 - Second VIRB filmed session - Nov. 7, 2018
# 15692 - Third VIRB filmed session - Nov. 9, 2018
# 15686 - Fourth VIRB filmed session - Nov. 11, 2018

ride_ids = [15218, 15669, 15692, 15686]

In [60]:
#Get urls for surf sessions in Scripps beach area that most likely correspond to surfing time intervals: 
url_45_60 = 'https://surf.smartfin.org/advanced_search/?northEastLat=32.6&northEastLon=-117.24&southWestLat=32.9&southWestLon=-117.33&dateTimeBegin=&dateTimeEnd=&timeZone=PDT&durationMin=45&durationMax=60&sensorTypeWave=true#searchResultsContainer'
url_61_80 = 'https://surf.smartfin.org/advanced_search/?northEastLat=32.6&northEastLon=-117.24&southWestLat=32.9&southWestLon=-117.33&dateTimeBegin=&dateTimeEnd=&timeZone=PDT&durationMin=61&durationMax=80&sensorTypeWave=true#searchResultsContainer'
url_81_100 = 'https://surf.smartfin.org/advanced_search/?northEastLat=32.6&northEastLon=-117.24&southWestLat=32.9&southWestLon=-117.33&dateTimeBegin=&dateTimeEnd=&timeZone=PDT&durationMin=81&durationMax=100&sensorTypeWave=true#searchResultsContainer'
url_101_130 = 'https://surf.smartfin.org/advanced_search/?northEastLat=32.6&northEastLon=-117.24&southWestLat=32.9&southWestLon=-117.33&dateTimeBegin=&dateTimeEnd=&timeZone=PDT&durationMin=101&durationMax=130&sensorTypeWave=true#searchResultsContainer'

urls = [url_45_60]
#urls = [url_45_60, url_61_80, url_81_100, url_101_130]

In [61]:
# Define a function that will parse rows from data tables on CDIP's website:
def parse_rows(row_number):
    name_string = ""
    name_list = []
    for t in tr_elements[row_number]:
        name = t.text_content()
        for i in name: 
            name_string += i
    #print(name_string)

    # Create a string from the values
    name_string = name_string.split(" ")
    
    # Remove all spaces from the list
    for i in name_string: 
        if len(i) > 0:
            name_list.append(i)
      
    # Ensure that time stays with 'Date (UTC)'' header
    name_list[0] = name_list[0] + " " +  name_list[1]
    name_list.pop(1)
    
    return name_list

In [62]:
# Iterate over each url and get all ride_ids located at that url: 
ride_ids = []
for u in urls: 
    
    # Create a handle, page, to handle the contents of the website
    page = requests.get(u)

    # Store the contents of the website under doc
    doc = lh.fromstring(page.content)

    # Parse data stored between <tr>..</tr> of HTML
    tr_elements = doc.xpath('//tr')

    # Retrieve all of the ride ids: 
    for j in range(1, len(tr_elements)):
        data = parse_rows(j)
        data = str(data[0]).strip("\n")
        if int(data) > 10000: 
            ride_ids.append(data)

print(len(ride_ids))
print(ride_ids[:20])
print(ride_ids[-10:])

194
[' 16345', ' 16318', ' 16300', ' 16289', ' 16278', ' 16211', ' 16197', ' 16194', ' 16191', ' 16169', ' 16167', ' 16137', ' 16135', ' 16134', ' 16124', ' 16115', ' 16099', ' 16098', ' 16088', ' 16076']
[' 10324', ' 10247', ' 10223', ' 10214', ' 10212', ' 10208', ' 10190', ' 10074', ' 10069', ' 10066']


In [63]:
# Fin ID Scraper: 

ride_url_base = 'http://surf.smartfin.org/ride/'
str_id_csv = 'img id="temperatureChart" class="chart" src="' # Look for this text in the HTML contents in fcn below

def read_csv_from_ride(ride): 
    ride_url = ride_url_base + str(ride)
    html_contents = requests.get(ride_url).text # Get contents of ride_url
    loc_csv_id = html_contents.find(str_id_csv) # Find CSV identifier 
    
    # Different based on whether user logged in with FB or Google
    offset_googleOAuth = [46, 114]
    offset_facebkOAuth = [46, 112]
    if html_contents[loc_csv_id+59] == 'f': # Facebook login
        off0 = offset_facebkOAuth[0]
        off1 = offset_facebkOAuth[1]
    else: # Google login
        off0 = offset_googleOAuth[0]
        off1 = offset_googleOAuth[1]
        
    csv_id_longstr = html_contents[loc_csv_id+off0:loc_csv_id+off1]
        
    # Stitch together full URL for CSV
        
    motion_csv_url = 'https://surf.smartfin.org/'+csv_id_longstr+'Motion.CSV'

    #print(motion_csv_url)

    motion_df_small = pd.read_csv(motion_csv_url)
    #print(len(motion_df_small))
        
        
    return motion_df_small

In [64]:
#ride_ids = ride_ids[:10]

In [65]:
#%% Fin ID scraper
# Input fin ID, get all ride IDs
# base URL to which we'll append given fin IDs
fin_url_base = 'http://surf.smartfin.org/fin/'

# Look for the following text in the HTML contents in fcn below
str_id_ride = 'rideId = \'' # backslash allows us to look for single quote
str_id_date = 'var date = \'' # backslash allows us to look for single quote

#%% Ride ID scraper
# Input ride ID, get ocean and motion CSVs
# Base URL to which we'll append given ride IDs
ride_url_base = 'https://surf.smartfin.org/ride/'

# Look for the following text in the HTML contents in fcn below
str_id_csv = 'img id="temperatureChart" class="chart" src="' 

def get_csv_from_ride_id(rid):
    # Build URL for each individual ride
    ride_url = ride_url_base+str(rid)
    #print(ride_url)
    
    # Get contents of ride_url
    html_contents = requests.get(ride_url).text
    
    # Find CSV identifier 
    loc_csv_id = html_contents.find(str_id_csv)
    
    # Different based on whether user logged in with FB or Google
    offset_googleOAuth = [46, 114]
    offset_facebkOAuth = [46, 112]
    if html_contents[loc_csv_id+59] == 'f': # Facebook login
        off0 = offset_facebkOAuth[0]
        off1 = offset_facebkOAuth[1]
    else: # Google login
        off0 = offset_googleOAuth[0]
        off1 = offset_googleOAuth[1]
        
    csv_id_longstr = html_contents[loc_csv_id+off0:loc_csv_id+off1]
        
    # Stitch together full URL for CSV
    if ("media" in csv_id_longstr) & ("Calibration" not in html_contents): # other junk URLs can exist and break everything
        
        ocean_csv_url = 'https://surf.smartfin.org/'+csv_id_longstr+'Ocean.CSV'
        motion_csv_url = 'https://surf.smartfin.org/'+csv_id_longstr+'Motion.CSV'
        
        print(motion_csv_url)
        
        # Go to ocean_csv_url and grab contents (theoretically, a CSV)
        ocean_df_small = pd.read_csv(ocean_csv_url, parse_dates = [0])
        elapsed_timedelta = (ocean_df_small['UTC']-ocean_df_small['UTC'][0])
        ocean_df_small['elapsed'] = elapsed_timedelta/np.timedelta64(1, 's')
        
        motion_df_small = pd.read_csv(motion_csv_url, parse_dates = [0])
        print(len(motion_df_small))
        
        # Reindex on timestamp if there are at least a few rows
        if len(ocean_df_small) > 1:
            ocean_df_small.set_index('UTC', drop = True, append = False, inplace = True)
            motion_df_small.set_index('UTC', drop = True, append = False, inplace = True)            
            
            #May need to change this sampling interval:
            sample_interval = '33ms'
            
            ocean_df_small_resample = ocean_df_small.resample(sample_interval).mean()
            motion_df_small_resample = motion_df_small.resample(sample_interval).mean()
            
            # No need to save many extra rows with no fix
            motion_df_small = motion_df_small[~np.isnan(motion_df_small.Latitude)]

            
    else:
        ocean_df_small_resample = pd.DataFrame() # empty DF just so something is returned
        motion_df_small_resample = pd.DataFrame() 
        print(len(motion_df_small_resample))
        return ocean_df_small_resample, motion_df_small_resample

In [66]:
# Fin ID Scraper: 

ride_url_base = 'http://surf.smartfin.org/ride/'
str_id_csv = 'img id="temperatureChart" class="chart" src="' # Look for this text in the HTML contents in fcn below

def read_csv_from_ride2(ride): 
    smartfin_url_base = 'http://surf.smartfin.org'
    ride_url = ride_url_base + str(ride)
    #html_page = requests.get(ride_url)
    driver = webdriver.Remote(service.service_url, options=chrome_options)
    driver.get(ride_url);
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    #print(soup)
    links = []
    for link in soup.findAll('a'):
        links.append(link)
    if len(links) > 8: 
        motion_csv_link = str(links[8])
        #print(links[8])
        motion_csv_link = motion_csv_link.split(" ")
        motion_csv_link = motion_csv_link[1]
        motion_csv_link = motion_csv_link.split("\"")
        motion_csv_link = motion_csv_link[1]

        #print(motion_csv_link)

        motion_csv_link = smartfin_url_base + motion_csv_link
        motion_df_small = pd.read_csv(motion_csv_link, parse_dates = [0])
        

        return 1, motion_df_small
    
    else: 
        
        return 0, 0

In [67]:
appended_motion_list = []
appended_multiIndex = [] # fin_id & ride_id used to identify each DataFrame

counter = 0
index = 1
for ride in ride_ids: 
    #ride = '16167'
    b, motion_df = read_csv_from_ride2(ride)
    
    # Check to make sure that a .CSV link was found for motion: 
    if b == 1: 
        counter += 1
        print(counter, len(motion_df))
        motion_df.set_index('UTC', drop = True, append = False, inplace = True)
        appended_multiIndex.append(str(ride)) # build list to be multiIndex of future DataFrame
        appended_motion_list.append(motion_df)
        
            
    print(index)
    index += 1

print("Total number of rides downloaded: ", counter)
df_keys = tuple(appended_multiIndex)
motion_df = pd.concat(appended_motion_list, keys = df_keys, names = ['ride_id'])

    

1 13087
1
2 15297
2
3 13971
3
4 11732
4
5 12705
5
6 14766
6


ParserError: Error tokenizing data. C error: Expected 1 fields in line 3, saw 2


In [ ]:
motion_df[:10] # 18 / 37 with 5 repeated requests, 18 / 37 with 10 repeated requests.  

In [ ]:
'''
appended_ocean_list = [] # list of DataFrames from original CSVs
appended_motion_list = []
appended_multiIndex = [] # fin_id & ride_id used to identify each DataFrame

print("Once the counter gets to ", len(ride_ids), " it will be done printing.")

## Nested loops (for each fin ID, find all ride IDs, then build a DataFrame from all ride CSVs)
## (Here, ride IDS are either ocean or motion dataframes)
count_good_fins = 0
    
# Loop over ride_ids and find CSVs
counter = 0
for rid in ride_ids:
    counter += 1
    if counter % 10 == 0:
        print(counter)
    try:
        new_motion_df = read_csv_from_ride(rid) # get given ride's CSV from its ride ID using function above
        new_motion_df.set_index('UTC', drop = True, append = False, inplace = True)            

        if not new_motion_df.empty: # Calibration rides, for example
            # Append only if DF isn't empty. There may be a better way to control empty DFs which are created above
            appended_multiIndex.append(str(rid)) # build list to be multiIndex of future DataFrame
            #appended_ocean_list.append(new_ocean_df)
            appended_motion_list.append(new_motion_df)
            #print("Ride data has been uploaded.")
            #print("Ride: ", rid, "data has been uploaded.")
            count_good_fins += 1
            #print("Ride ", rid, "worked!")    
     
    except: 
        print("Ride ", rid, "threw an exception!")    

#%% Build the "Master" DataFrame

# appended_ocean_df.summary()

df_keys = tuple(appended_multiIndex) # keys gotta be a tuple, a list which data in it cannot be changed
#ocean_df = pd.concat(appended_ocean_list, keys = df_keys, names=['ride_id'])
motion_df = pd.concat(appended_motion_list, keys = df_keys, names = ['ride_id'])


##Here, maybe just use info from the motion_df and don't worry about ocean_df data for now.
##If you do want ocean_df data, look at how Phil was getting it from "July 10th and 11th Calibration" jupyter notebook file.
#print(motion_df)
#print(appended_motion_list)
print("Done.")
'''

In [ ]:
# View data in a python dataframe: 
print(len(motion_df))
motion_df[0:10]

# I think that we're resampling each surf session at too small of a rate? 
# Why do we have so little data? 

#Sampling interval of 33ms: 3,290,682 initial data points, 438,099 final data points (adding .033 to UTC time)
#Sampling interval of 20ms: 5,429,605 initial data points, 438,099 final data points (adding .020 to UTC time)

In [51]:
motion_df.shape

(255382, 12)

In [121]:
# Drop latitude/longitude columns and remove NaN rows: 
motion_df = motion_df.drop(["Latitude", "Longitude"], axis=1)
motion_df = motion_df.dropna(axis=0, how='any')
print(len(motion_df))
motion_df[0:20]

621377


Time  IMU A1  IMU A2  IMU A3  \
ride_id UTC                                                                    
 16345  2019-11-19 15:02:50.090000+00:00  3888630399   502.0    41.0    60.0   
        2019-11-19 15:02:50.340000+00:00  3888630649   501.0    37.0    63.0   
        2019-11-19 15:02:50.590000+00:00  3888630899   503.0    36.0    59.0   
        2019-11-19 15:02:50.841000+00:00  3888631150   500.0    35.0    62.0   
        2019-11-19 15:02:51.092000+00:00  3888631401   501.0    33.0    61.0   
        2019-11-19 15:02:51.342000+00:00  3888631651   500.0    34.0    63.0   
        2019-11-19 15:02:51.593000+00:00  3888631902   501.0    34.0    61.0   
        2019-11-19 15:02:51.843000+00:00  3888632152   502.0    35.0    62.0   
        2019-11-19 15:02:52.094000+00:00  3888632403   502.0    23.0    61.0   
        2019-11-19 15:02:52.344000+00:00  3888632653   503.0    21.0    61.0   
        2019-11-19 15:02:52.594000+00:00  3888632903   503.0    24.0    60.0   
        2019-11-19 15:02:52.846000+00:00  3888633155   501.0    23.0    61.0   
        2019-11-19 15:02:53.096000+00:00  3888633405   503.0    18.0    60.0   
        2019-11-19 15:02:53.348000+00:00  3888633657   503.0    15.0    60.0   
        2019-11-19 15:02:53.598000+00:00  3888633907   502.0    16.0    60.0   
        2019-11-19 15:02:53.848000+00:00  3888634157   504.0    15.0    60.0   
        2019-11-19 15:02:54.099000+00:00  3888634408   503.0    10.0    59.0   
        2019-11-19 15:02:54.340000+00:00  3888634649   503.0     6.0    61.0   
        2019-11-19 15:02:54.592000+00:00  3888634901   502.0     4.0    58.0   
        2019-11-19 15:02:54.842000+00:00  3888635151   503.0    10.0    60.0   

                                          IMU G1  IMU G2  IMU G3  IMU M1  \
ride_id UTC                                                                
 16345  2019-11-19 15:02:50.090000+00:00   -18.0   -10.0    -1.0    32.0   
        2019-11-19 15:02:50.340000+00:00   -18.0   -10.0     0.0    39.0   
        2019-11-19 15:02:50.590000+00:00   -18.0    -9.0     0.0    38.0   
        2019-11-19 15:02:50.841000+00:00   -19.0    -9.0     3.0    37.0   
        2019-11-19 15:02:51.092000+00:00   -18.0   -10.0     0.0    38.0   
        2019-11-19 15:02:51.342000+00:00   -18.0   -10.0     0.0    30.0   
        2019-11-19 15:02:51.593000+00:00   -19.0    -9.0     3.0    39.0   
        2019-11-19 15:02:51.843000+00:00   -19.0   -11.0    11.0    41.0   
        2019-11-19 15:02:52.094000+00:00   -19.0   -12.0    16.0    42.0   
        2019-11-19 15:02:52.344000+00:00   -20.0   -10.0     1.0    43.0   
        2019-11-19 15:02:52.594000+00:00   -19.0    -9.0    -3.0    43.0   
        2019-11-19 15:02:52.846000+00:00   -19.0    -9.0     2.0    34.0   
        2019-11-19 15:02:53.096000+00:00   -19.0   -10.0     8.0    32.0   
        2019-11-19 15:02:53.348000+00:00   -19.0   -10.0     4.0    29.0   
        2019-11-19 15:02:53.598000+00:00   -19.0    -9.0     4.0    40.0   
        2019-11-19 15:02:53.848000+00:00   -19.0   -10.0     8.0    32.0   
        2019-11-19 15:02:54.099000+00:00   -19.0    -9.0    11.0    29.0   
        2019-11-19 15:02:54.340000+00:00   -19.0    -9.0     4.0    31.0   
        2019-11-19 15:02:54.592000+00:00   -19.0    -9.0    -7.0    41.0   
        2019-11-19 15:02:54.842000+00:00   -19.0    -9.0    -8.0    25.0   

                                          IMU M2  IMU M3  
ride_id UTC                                               
 16345  2019-11-19 15:02:50.090000+00:00  -110.0  -468.0  
        2019-11-19 15:02:50.340000+00:00  -115.0  -461.0  
        2019-11-19 15:02:50.590000+00:00  -110.0  -462.0  
        2019-11-19 15:02:50.841000+00:00  -111.0  -457.0  
        2019-11-19 15:02:51.092000+00:00  -116.0  -460.0  
        2019-11-19 15:02:51.342000+00:00  -108.0  -460.0  
        2019-11-19 15:02:51.593000+00:00  -115.0  -453.0  
        2019-11-19 15:02:51.843000+00:00  -113.0  -465.0  
        2019-11-19 15:02

### Scrape data from same-day CDIP and add it as an additional column to the dataframe:


In [122]:
# Format should be appending year, month
# Example: Sept. 2019 -> 201909
def create_url_string(year, month):
    url='https://cdip.ucsd.edu/themes/cdip?tz=UTC&numcolorbands=10&palette=cdip_classic&zoom=auto&ll_fmt=dm&high=6.096&r=999&un=1&pb=1&d2=p70&u2=s:201:st:1:v:parameter:dt:'
    if 2014 <= int(year) <= 2019:
        year = year
    if 1 <= int(month) <= 12 and len(month) == 2:
        month = month
    url += year
    url += month
    return url 

In [123]:
# For each row in the dataframe, parse the index ('UTC' column) to get the year, month information. 
motion_df.columns

dates = []
times = []
for row in motion_df.index:
    time = str(row[1])
    date = time.split(" ")
    time = date[1]
    date = date[0]
    time = time.split(".")
    time = time[0]
    time = time.split(":")
    time = time[0] + ":" + time[1]
    times.append(time)
    dates.append(date)
print(dates[0:10], times[0:10])

motion_df["Date"] = dates
motion_df["Time"] = times

['2019-11-19', '2019-11-19', '2019-11-19', '2019-11-19', '2019-11-19', '2019-11-19', '2019-11-19', '2019-11-19', '2019-11-19', '2019-11-19'] ['15:02', '15:02', '15:02', '15:02', '15:02', '15:02', '15:02', '15:02', '15:02', '15:02']


In [124]:
motion_df[0:10]

Time  IMU A1  IMU A2  IMU A3  \
ride_id UTC                                                               
 16345  2019-11-19 15:02:50.090000+00:00  15:02   502.0    41.0    60.0   
        2019-11-19 15:02:50.340000+00:00  15:02   501.0    37.0    63.0   
        2019-11-19 15:02:50.590000+00:00  15:02   503.0    36.0    59.0   
        2019-11-19 15:02:50.841000+00:00  15:02   500.0    35.0    62.0   
        2019-11-19 15:02:51.092000+00:00  15:02   501.0    33.0    61.0   
        2019-11-19 15:02:51.342000+00:00  15:02   500.0    34.0    63.0   
        2019-11-19 15:02:51.593000+00:00  15:02   501.0    34.0    61.0   
        2019-11-19 15:02:51.843000+00:00  15:02   502.0    35.0    62.0   
        2019-11-19 15:02:52.094000+00:00  15:02   502.0    23.0    61.0   
        2019-11-19 15:02:52.344000+00:00  15:02   503.0    21.0    61.0   

                                          IMU G1  IMU G2  IMU G3  IMU M1  \
ride_id UTC                                                                
 16345  2019-11-19 15:02:50.090000+00:00   -18.0   -10.0    -1.0    32.0   
        2019-11-19 15:02:50.340000+00:00   -18.0   -10.0     0.0    39.0   
        2019-11-19 15:02:50.590000+00:00   -18.0    -9.0     0.0    38.0   
        2019-11-19 15:02:50.841000+00:00   -19.0    -9.0     3.0    37.0   
        2019-11-19 15:02:51.092000+00:00   -18.0   -10.0     0.0    38.0   
        2019-11-19 15:02:51.342000+00:00   -18.0   -10.0     0.0    30.0   
        2019-11-19 15:02:51.593000+00:00   -19.0    -9.0     3.0    39.0   
        2019-11-19 15:02:51.843000+00:00   -19.0   -11.0    11.0    41.0   
        2019-11-19 15:02:52.094000+00:00   -19.0   -12.0    16.0    42.0   
        2019-11-19 15:02:52.344000+00:00   -20.0   -10.0     1.0    43.0   

                                          IMU M2  IMU M3        Date  
ride_id UTC                                                           
 16345  2019-11-19 15:02:50.090000+00:00  -110.0  -468.0  2019-11-19  
        2019-11-19 15:02:50.340000+00:00  -115.0  -461.0  2019-11-19  
        2019-11-19 15:02:50.590000+00:00  -110.0  -462.0  2019-11-19  
        2019-11-19 15:02:50.841000+00:00  -111.0  -457.0  2019-11-19  
        2019-11-19 15:02:51.092000+00:00  -116.0  -460.0  2019-11-19  
        2019-11-19 15:02:51.342000+00:00  -108.0  -460.0  2019-11-19  
        2019-11-19 15:02:51.593000+00:00  -115.0  -453.0  2019-11-19  
        2019-11-19 15:02:51.843000+00:00  -113.0  -465.0  2019-11-19  
        2019-11-19 15:02:52.094000+00:00  -118.0  -454.0  2019-11-19  
        2019-11-19 15:02:52.344000+00:00  -111.0  -457.0  2019-11-19

In [125]:
# Define a function that will find the closest time on the CDIP table to the surf ride's time: 
def find_closest_single_time(date, time_utc):
    time_list = []
    
    # Iterate over each date
    for x in df_data['Date(UTC)']:
        a = x.split(" ")
        if date == a[0]:
            a = str(a[1]).split(':')
            y = str(time_utc).split(":")
            #print(a)
            #print(y)
            
            # Find the closest timestamp (less than 15 minutes away)
            time1 = int(a[0])*60 + int(a[1])
            time2 = int(y[0])*60 + int(y[1])
            
            if abs(time1 - time2) <= 15:
                time_string = str(a[0]) + ":" + str(a[1])
                time_list.append(time_string)
           
    return time_list

### Create the output labels from CDIP data for each Smartfin timestamp: 


In [126]:
# Create the output labels from CDIP data for each Smartfin timestamp: 
old_url = 'old_url'
old_date = 'old_date'
old_time = 'xx:xx'

wave_heights = []
wave_periods = []
wave_directions = []
for d, time in zip(dates,times):
    di = d.split("-")
    year = di[0]
    month = di[1]
    url = create_url_string(year, month)
    
    #We need to load a new table since we have a new surf ride session:
    if (url != old_url): 
        old_url = url
        # Create a handle, page, to handle the contents of the website
        page = requests.get(url)

        # Store the contents of the website under doc
        doc = lh.fromstring(page.content)

        # Parse data stored between <tr>..</tr> of HTML
        tr_elements = doc.xpath('//tr')

        # Need to drop the first 3 rows since they aren't in the table
        tr_elements = tr_elements[3:]
        
        # Parse the first row as the header
        tr_elements = doc.xpath('//tr')

        # Create empty list
        headers = []
        i = 0

        # For each row, store each first element (header) and an empty list
        for t in tr_elements[3]:
            i+=1
            name=t.text_content()
            #print(name)
            headers.append(name)

        #print(headers)

        # Create a Pandas dataframe: 
        data_list = []

        #Since out first row is the header, data is stored on the second row onwards
        for j in range(4, len(tr_elements)):
            data = parse_rows(j)
            data_list.append(data)

        df_data = pd.DataFrame(data_list, columns=headers)
        #print(df_data[:10])
        
        #Find the closest time in the CDIP data table that corresponds to the Smartfin data: 
        date = d #YYYY-MM-DD format
        print("date, time ", date, time)
        time_list = find_closest_single_time(date, time)
        print("time list ", time_list)
        
        # Now compute the average significant wave height for that time period:
        length = df_data['Date(UTC)'].size
        wave_height_list = []
        wave_period_list = []
        wave_direction_list = []
        for time in time_list: 
            date_data = date + " " + time
            for i in range(0, length):
                if df_data['Date(UTC)'][i] == date_data:
                    print("height ", df_data['Hs(ft)'][i])
                    wave_height_list.append(float(df_data['Hs(ft)'][i]))
                    wave_period_list.append(float(df_data['Tp(s)'][i]))
                    wave_direction_list.append(float(df_data['Dp(deg)'][i]))
                    
        # Throws error when date doesn't exist on CDIP data (ex: 2018-01-18)
        if len(wave_height_list) == 0 or len(wave_period_list) == 0 or len(wave_direction_list) == 0: 
            avg_wave_height = np.nan
            avg_wave_period = np.nan
            avg_wave_direction = np.nan
            
        else: 
            #print(wave_height_list)
            avg_wave_height = sum(wave_height_list)/len(wave_height_list)
            avg_wave_period = sum(wave_period_list)/len(wave_period_list)
            avg_wave_direction = sum(wave_direction_list)/len(wave_direction_list)
            #print(avg_wave_height)
            #print(avg_wave_period)
            #print(avg_wave_direction) 
            
    # Same date/month but maybe a different day or time:
    elif old_date != date or old_time[3] != time[3]:
        date = d #YYYY-MM-DD format
        #print(date, time)
        time_list = find_closest_single_time(date, time)
        #print(time_list)
        
        old_date = date
        old_time = time
        
        # Now compute the average significant wave height for that time period:
        length = df_data['Date(UTC)'].size
        wave_height_list = []
        wave_period_list = []
        wave_direction_list = []
        for time in time_list: 
            date_data = date + " " + time
            for i in range(0, length):
                if df_data['Date(UTC)'][i] == date_data:
                    #print(df_data['Hs(ft)'][i])
                    wave_height_list.append(float(df_data['Hs(ft)'][i]))
                    wave_period_list.append(float(df_data['Tp(s)'][i]))
                    wave_direction_list.append(float(df_data['Dp(deg)'][i]))
                    
      
        # Throws error when date doesn't exist on CDIP data (ex: 2018-01-18)
        if len(wave_height_list) == 0 or len(wave_period_list) == 0 or len(wave_direction_list) == 0: 
            avg_wave_height = np.nan
            avg_wave_period = np.nan
            avg_wave_direction = np.nan
            
        else: 
            #print(wave_height_list)
            avg_wave_height = sum(wave_height_list)/len(wave_height_list)
            avg_wave_period = sum(wave_period_list)/len(wave_period_list)
            avg_wave_direction = sum(wave_direction_list)/len(wave_direction_list)
            #print(avg_wave_height)
            #print(avg_wave_period)
            #print(avg_wave_direction)    

    # If they have the same url, date, and time then they will have the same value computed. 
    wave_heights.append(avg_wave_height)
    wave_periods.append(avg_wave_period)
    wave_directions.append(avg_wave_direction)

date, time  2019-11-19 15:02
time list  ['15:02']
height  1.74
date, time  2019-10-24 17:37
time list  ['17:32']
height  2.00
date, time  2019-08-08 16:15
time list  ['16:02']
height  1.87
date, time  2019-07-24 16:22
time list  ['16:32']
height  1.94
date, time  2019-02-26 21:06
time list  ['21:02']
height  0.92
date, time  2018-11-11 20:34
time list  ['20:30']
height  1.02
date, time  2017-10-21 20:46
time list  ['20:51']
height  4.86
date, time  2019-10-26 19:04
time list  ['19:02']
height  1.25
date, time  2019-09-24 15:28
time list  ['15:32']
height  1.80
date, time  2019-02-25 01:31
time list  ['01:32']
height  1.71
date, time  2018-01-18 14:17
time list  []
date, time  2017-05-03 18:40
time list  ['18:51']
height  1.80
date, time  2019-09-20 16:15
time list  ['16:02']
height  3.15
date, time  2018-11-09 19:16
time list  ['19:30']
height  1.44
date, time  2019-09-18 16:04
time list  ['16:02']
height  2.49
date, time  2019-02-08 17:17
time list  ['17:32', '17:02']
height  1.38
hei

In [127]:
print(len(dates))
print(len(wave_heights))
print(len(wave_periods))
print(len(wave_directions))


# Check to make sure different wave heights were appended throughout the dataframe: 
motion_df["Hs(ft)"] = wave_heights
motion_df["Tp(s)"] = wave_periods
motion_df["Dp(deg)"] = wave_directions

for x in range(0, 60000, 10000):
    print(motion_df.iloc[[x]]['Hs(ft)'])
    print(motion_df.iloc[[x]]['Tp(s)'])
    print(motion_df.iloc[[x]]['Dp(deg)'])

motion_df[0:5]

621377
621377
621377
621377
ride_id  UTC                             
 16345   2019-11-19 15:02:50.090000+00:00    1.74
Name: Hs(ft), dtype: float64
ride_id  UTC                             
 16345   2019-11-19 15:02:50.090000+00:00    10.53
Name: Tp(s), dtype: float64
ride_id  UTC                             
 16345   2019-11-19 15:02:50.090000+00:00    285.0
Name: Dp(deg), dtype: float64
ride_id  UTC                             
 16345   2019-11-19 15:43:24.942000+00:00    1.77
Name: Hs(ft), dtype: float64
ride_id  UTC                             
 16345   2019-11-19 15:43:24.942000+00:00    11.76
Name: Tp(s), dtype: float64
ride_id  UTC                             
 16345   2019-11-19 15:43:24.942000+00:00    282.0
Name: Dp(deg), dtype: float64
ride_id  UTC                             
 16318   2019-11-09 21:07:19.804000+00:00    1.97
Name: Hs(ft), dtype: float64
ride_id  UTC                             
 16318   2019-11-09 21:07:19.804000+00:00    5.26
Name: Tp(s), dtype: float64
r

Time  IMU A1  IMU A2  IMU A3  \
ride_id UTC                                                               
 16345  2019-11-19 15:02:50.090000+00:00  15:02   502.0    41.0    60.0   
        2019-11-19 15:02:50.340000+00:00  15:02   501.0    37.0    63.0   
        2019-11-19 15:02:50.590000+00:00  15:02   503.0    36.0    59.0   
        2019-11-19 15:02:50.841000+00:00  15:02   500.0    35.0    62.0   
        2019-11-19 15:02:51.092000+00:00  15:02   501.0    33.0    61.0   

                                          IMU G1  IMU G2  IMU G3  IMU M1  \
ride_id UTC                                                                
 16345  2019-11-19 15:02:50.090000+00:00   -18.0   -10.0    -1.0    32.0   
        2019-11-19 15:02:50.340000+00:00   -18.0   -10.0     0.0    39.0   
        2019-11-19 15:02:50.590000+00:00   -18.0    -9.0     0.0    38.0   
        2019-11-19 15:02:50.841000+00:00   -19.0    -9.0     3.0    37.0   
        2019-11-19 15:02:51.092000+00:00   -18.0   -10.0     0.0    38.0   

                                          IMU M2  IMU M3        Date  Hs(ft)  \
ride_id UTC                                                                    
 16345  2019-11-19 15:02:50.090000+00:00  -110.0  -468.0  2019-11-19    1.74   
        2019-11-19 15:02:50.340000+00:00  -115.0  -461.0  2019-11-19    1.74   
        2019-11-19 15:02:50.590000+00:00  -110.0  -462.0  2019-11-19    1.74   
        2019-11-19 15:02:50.841000+00:00  -111.0  -457.0  2019-11-19    1.74   
        2019-11-19 15:02:51.092000+00:00  -116.0  -460.0  2019-11-19    1.74   

                                          Tp(s)  Dp(deg)  
ride_id UTC                                               
 16345  2019-11-19 15:02:50.090000+00:00  10.53    285.0  
        2019-11-19 15:02:50.340000+00:00  10.53    285.0  
        2019-11-19 15:02:50.590000+00:00  10.53    285.0  
        2019-11-19 15:02:50.841000+00:00  10.53    285.0  
        2019-11-19 15:02:51.092000+00:00  10.53    285.0

In [128]:
# Now drop all of the nan values that I introduced earlier: 
motion_df = motion_df.dropna(axis=0, how='any')
print(len(motion_df))
motion_df[0:5]

508034


Time  IMU A1  IMU A2  IMU A3  \
ride_id UTC                                                               
 16345  2019-11-19 15:02:50.090000+00:00  15:02   502.0    41.0    60.0   
        2019-11-19 15:02:50.340000+00:00  15:02   501.0    37.0    63.0   
        2019-11-19 15:02:50.590000+00:00  15:02   503.0    36.0    59.0   
        2019-11-19 15:02:50.841000+00:00  15:02   500.0    35.0    62.0   
        2019-11-19 15:02:51.092000+00:00  15:02   501.0    33.0    61.0   

                                          IMU G1  IMU G2  IMU G3  IMU M1  \
ride_id UTC                                                                
 16345  2019-11-19 15:02:50.090000+00:00   -18.0   -10.0    -1.0    32.0   
        2019-11-19 15:02:50.340000+00:00   -18.0   -10.0     0.0    39.0   
        2019-11-19 15:02:50.590000+00:00   -18.0    -9.0     0.0    38.0   
        2019-11-19 15:02:50.841000+00:00   -19.0    -9.0     3.0    37.0   
        2019-11-19 15:02:51.092000+00:00   -18.0   -10.0     0.0    38.0   

                                          IMU M2  IMU M3        Date  Hs(ft)  \
ride_id UTC                                                                    
 16345  2019-11-19 15:02:50.090000+00:00  -110.0  -468.0  2019-11-19    1.74   
        2019-11-19 15:02:50.340000+00:00  -115.0  -461.0  2019-11-19    1.74   
        2019-11-19 15:02:50.590000+00:00  -110.0  -462.0  2019-11-19    1.74   
        2019-11-19 15:02:50.841000+00:00  -111.0  -457.0  2019-11-19    1.74   
        2019-11-19 15:02:51.092000+00:00  -116.0  -460.0  2019-11-19    1.74   

                                          Tp(s)  Dp(deg)  
ride_id UTC                                               
 16345  2019-11-19 15:02:50.090000+00:00  10.53    285.0  
        2019-11-19 15:02:50.340000+00:00  10.53    285.0  
        2019-11-19 15:02:50.590000+00:00  10.53    285.0  
        2019-11-19 15:02:50.841000+00:00  10.53    285.0  
        2019-11-19 15:02:51.092000+00:00  10.53    285.0

In [129]:
# We're collecting IMU data at 3-4 Hz here. 
# We're using 4 surf sessions and we have 60,000 data points. 
# If we only calculated Hs once per minute then we would have 330 data points. 

# Instead of looking at significant wave height, we could look at wave direction 
# or we could look at Hs, dir, and period and see which one we predict best. 



In [130]:
# Convert raw IMU values to real values: 
g_const = 512 #Raw acceleration constant 512 = 1g (accelerometer's measured force due to gravity)
gravity = 9.80665 #Approximate measurement for gravity

# Correct the IMU Acceleration columns into units of meters
# Dividing by 512 is equivalent to muliplying by 4 to correct the bit shifting by 2 places and dividing by 2048 to convert bits to G's
# Multiplying by the 9.81 afterwards is simply to convert G's into m/s^2
motion_df['IMU A1'] = motion_df['IMU A1'].apply(lambda x: x / g_const * gravity)
motion_df['IMU A2'] = motion_df['IMU A2'].apply(lambda x: x / g_const * gravity)
motion_df['IMU A3'] = motion_df['IMU A3'].apply(lambda x: x / g_const * gravity)

In [131]:
# Gyroscopic Rotation converts to deg/s
gyro_const = 8.2 # Raw gyrscope constant 8.2 bits per degree
motion_df['IMU G1'] = motion_df['IMU G1'].apply(lambda x: x / gyro_const)
motion_df['IMU G2'] = motion_df['IMU G2'].apply(lambda x: x / gyro_const)
motion_df['IMU G3'] = motion_df['IMU G3'].apply(lambda x: x / gyro_const)

In [132]:
# Magnetometer values
# Offset variables help in recentering the magnetic data in order to define direction and use trig functions
'''
M1_offset_var = 219.786
M2_offset_var = 180
M3_offset_var = 280

motion_df['IMU M1'] = motion_df['IMU M1'].apply(lambda x: x - M1_offset_var)
motion_df['IMU M2'] = motion_df['IMU M2'].apply(lambda x: x - M2_offset_var)
motion_df['IMU M3'] = motion_df['IMU M3'].apply(lambda x: x - M3_offset_var)
'''

"\nM1_offset_var = 219.786\nM2_offset_var = 180\nM3_offset_var = 280\n\nmotion_df['IMU M1'] = motion_df['IMU M1'].apply(lambda x: x - M1_offset_var)\nmotion_df['IMU M2'] = motion_df['IMU M2'].apply(lambda x: x - M2_offset_var)\nmotion_df['IMU M3'] = motion_df['IMU M3'].apply(lambda x: x - M3_offset_var)\n"

In [133]:
print(len(motion_df))
motion_df[:20]

508034


Time    IMU A1    IMU A2    IMU A3  \
ride_id UTC                                                                     
 16345  2019-11-19 15:02:50.090000+00:00  15:02  9.615114  0.785298  1.149217   
        2019-11-19 15:02:50.340000+00:00  15:02  9.595960  0.708684  1.206678   
        2019-11-19 15:02:50.590000+00:00  15:02  9.634267  0.689530  1.130063   
        2019-11-19 15:02:50.841000+00:00  15:02  9.576807  0.670376  1.187524   
        2019-11-19 15:02:51.092000+00:00  15:02  9.595960  0.632069  1.168370   
        2019-11-19 15:02:51.342000+00:00  15:02  9.576807  0.651223  1.206678   
        2019-11-19 15:02:51.593000+00:00  15:02  9.595960  0.651223  1.168370   
        2019-11-19 15:02:51.843000+00:00  15:02  9.615114  0.670376  1.187524   
        2019-11-19 15:02:52.094000+00:00  15:02  9.615114  0.440533  1.168370   
        2019-11-19 15:02:52.344000+00:00  15:02  9.634267  0.402226  1.168370   
        2019-11-19 15:02:52.594000+00:00  15:02  9.634267  0.459687  1.149217   
        2019-11-19 15:02:52.846000+00:00  15:02  9.595960  0.440533  1.168370   
        2019-11-19 15:02:53.096000+00:00  15:02  9.634267  0.344765  1.149217   
        2019-11-19 15:02:53.348000+00:00  15:02  9.634267  0.287304  1.149217   
        2019-11-19 15:02:53.598000+00:00  15:02  9.615114  0.306458  1.149217   
        2019-11-19 15:02:53.848000+00:00  15:02  9.653421  0.287304  1.149217   
        2019-11-19 15:02:54.099000+00:00  15:02  9.634267  0.191536  1.130063   
        2019-11-19 15:02:54.340000+00:00  15:02  9.634267  0.114922  1.168370   
        2019-11-19 15:02:54.592000+00:00  15:02  9.615114  0.076614  1.110910   
        2019-11-19 15:02:54.842000+00:00  15:02  9.634267  0.191536  1.149217   

                                            IMU G1    IMU G2    IMU G3  \
ride_id UTC                                                              
 16345  2019-11-19 15:02:50.090000+00:00 -2.195122 -1.219512 -0.121951   
        2019-11-19 15:02:50.340000+00:00 -2.195122 -1.219512  0.000000   
        2019-11-19 15:02:50.590000+00:00 -2.195122 -1.097561  0.000000   
        2019-11-19 15:02:50.841000+00:00 -2.317073 -1.097561  0.365854   
        2019-11-19 15:02:51.092000+00:00 -2.195122 -1.219512  0.000000   
        2019-11-19 15:02:51.342000+00:00 -2.195122 -1.219512  0.000000   
        2019-11-19 15:02:51.593000+00:00 -2.317073 -1.097561  0.365854   
        2019-11-19 15:02:51.843000+00:00 -2.317073 -1.341463  1.341463   
        2019-11-19 15:02:52.094000+00:00 -2.317073 -1.463415  1.951220   
        2019-11-19 15:02:52.344000+00:00 -2.439024 -1.219512  0.121951   
        2019-11-19 15:02:52.594000+00:00 -2.317073 -1.097561 -0.365854   
        2019-11-19 15:02:52.846000+00:00 -2.317073 -1.097561  0.243902   
        2019-11-19 15:02:53.096000+00:00 -2.317073 -1.219512  0.975610   
        2019-11-19 15:02:53.348000+00:00 -2.317073 -1.219512  0.487805   
        2019-11-19 15:02:53.598000+00:00 -2.317073 -1.097561  0.487805   
        2019-11-19 15:02:53.848000+00:00 -2.317073 -1.219512  0.975610   
        2019-11-19 15:02:54.099000+00:00 -2.317073 -1.097561  1.341463   
        2019-11-19 15:02:54.340000+00:00 -2.317073 -1.097561  0.487805   
        2019-11-19 15:02:54.592000+00:00 -2.317073 -1.097561 -0.853659   
        2019-11-19 15:02:54.842000+00:00 -2.317073 -1.097561 -0.975610   

                                          IMU M1  IMU M2  IMU M3        Date  \
ride_id UTC                                                                    
 16345  2019-11-19 15:02:50.090000+00:00    32.0  -110.0  -468.0  2019-11-19   
        2019-11-19 15:02:50.340000+00:00    39.0  -115.0  -461.0  2019-11-19   
        2019-11-19 15:02:50.590000+00:00    38.0  -110.0  -462.0  2019-11-19   
        2019-11-19 15:02:50.841000+00:00    37.0  -111.0  -457.0  2019-11-19   
        2019-11-19 15:02:51.092000+00:00    38.0  -116.0  -460.0  2019-11-19   
        2019-11-19 15:02:51.342000+00:00    30.0  -108.0  -460.0  2019-

In [134]:
# After converting all to real values, try to export data to .CSV so everyone else doesn't have to webscrape it:
#motion_df.to_csv('CSE258_A2_Data.csv')

In [135]:
#motion_df.to_csv('CSE258_A2_Data2.csv')